In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46793
2,Huelva,Confirmados PDIA 14 días,545
3,Huelva,Tasa PDIA 14 días,"106,20262291248514"
4,Huelva,Confirmados PDIA 7 días,300
5,Huelva,Tasa PDIA 7 dias,"58,46015940136797"
6,Huelva,Total Confirmados,46996
7,Huelva,Curados,45541
8,Huelva,Fallecidos,421


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46793.0


/tmp/ipykernel_85310/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12902.0


/tmp/ipykernel_85310/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_85310/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_85310/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_85310/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1251 personas en los últimos 7 días 

Un positivo PCR cada 745 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46793.0,300.0,545.0,513170.0,58.460159,106.202623,65.0
Huelva-Costa,27666.0,177.0,315.0,289548.0,61.129761,108.790252,43.0
Huelva (capital),12902.0,115.0,193.0,143837.0,79.951612,134.179662,32.0
Condado-Campiña,14569.0,109.0,192.0,156231.0,69.768484,122.894944,22.0
Palos de la Frontera,1130.0,30.0,56.0,11742.0,255.493102,476.920456,7.0
Moguer,2065.0,21.0,29.0,21867.0,96.035121,132.619930,5.0
Aljaraque,1657.0,8.0,13.0,21474.0,37.254354,60.538325,4.0
Cartaya,2194.0,32.0,62.0,20083.0,159.338744,308.718817,4.0
Bollullos Par del Condado,1329.0,12.0,31.0,14387.0,83.408633,215.472301,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,59.0,1.0,6.0,713.0,140.252454,841.514727,0.0
Palos de la Frontera,1130.0,30.0,56.0,11742.0,255.493102,476.920456,7.0
Granada de Río-Tinto (La),6.0,1.0,1.0,243.0,411.522634,411.522634,0.0
Almonaster la Real,92.0,1.0,6.0,1788.0,55.928412,335.570470,0.0
Cartaya,2194.0,32.0,62.0,20083.0,159.338744,308.718817,4.0
Bollullos Par del Condado,1329.0,12.0,31.0,14387.0,83.408633,215.472301,3.0
San Bartolomé de la Torre,395.0,7.0,7.0,3761.0,186.120713,186.120713,1.0
Fuenteheridos,14.0,1.0,1.0,709.0,141.043724,141.043724,0.0
Palma del Condado (La),1451.0,11.0,15.0,10801.0,101.842422,138.876030,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almonaster la Real,92.0,1.0,6.0,1788.0,55.928412,335.570470,0.0,0.166667
Campofrío,59.0,1.0,6.0,713.0,140.252454,841.514727,0.0,0.166667
Lepe,3056.0,2.0,9.0,27880.0,7.173601,32.281205,0.0,0.222222
Sierra de Huelva-Andévalo Central,4130.0,13.0,36.0,67391.0,19.290410,53.419596,0.0,0.361111
Bollullos Par del Condado,1329.0,12.0,31.0,14387.0,83.408633,215.472301,3.0,0.387097
Almonte,2356.0,10.0,25.0,24507.0,40.804668,102.011670,1.0,0.400000
Valverde del Camino,864.0,2.0,5.0,12750.0,15.686275,39.215686,0.0,0.400000
Isla Cristina,3191.0,13.0,28.0,21393.0,60.767541,130.883934,2.0,0.464286
Rociana del Condado,676.0,1.0,2.0,7939.0,12.596045,25.192090,0.0,0.500000
